In [1]:
# Author: Endre Stølsvik, around 2017-04-08, based on first googlings about TensorFlow

import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [2]:
# Hyperparameters
hidden1_size = 800
hidden2_size = 800
epochs = 2500
mini_batch_size = 50
keep_prob_input = 0.8
keep_prob_hidden = 0.5
learning_rate = 0.0005

In [3]:
mnist = input_data.read_data_sets("tmp", one_hot=True)

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


In [4]:
# Define Neural Network

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
  return tf.Variable(tf.constant(0.1, shape=shape))

tf_keep_prob_input = tf.placeholder(tf.float32)
tf_keep_prob_hidden = tf.placeholder(tf.float32)

input = tf.placeholder(tf.float32, [None, 28*28])
input_dropped = tf.nn.dropout(input, tf_keep_prob_input)

layer1_w = weight_variable([28*28, hidden1_size])
layer1_b = bias_variable([hidden1_size])
layer1 = tf.matmul(input_dropped, layer1_w) + layer1_b
layer1 = tf.nn.elu(layer1)
layer1 = tf.nn.dropout(layer1, tf_keep_prob_hidden)

layer2_w = weight_variable([hidden1_size, hidden2_size])
layer2_b = bias_variable([hidden2_size])
layer2 = tf.matmul(layer1, layer2_w) + layer2_b
layer2 = tf.nn.elu(layer2)
layer2 = tf.nn.dropout(layer2, tf_keep_prob_hidden)

# Layer3 == Output layer
# Notice: Raw values; No activation/transfer function (i.e. Sigmoid, ReLU), nor dropout
layer3_w = weight_variable([hidden2_size, 10])
layer3_b = bias_variable([10])
layer3 = tf.matmul(layer2, layer3_w) + layer3_b

# Prediction = output
prediction = layer3

# Labels - "factual" - about the current input
label = tf.placeholder(tf.float32, [None, 10])

In [5]:
# Define Training: Loss/Cost and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=label))
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(cost)

In [6]:
# Define Test/Evaluate: Accuracy: Fraction right predictions
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# Create TensorFlow Session
s = tf.Session()
# .. initialize TensorFlow variables in Session
s.run(tf.global_variables_initializer())

In [8]:
# Train Neural Network

mini_batches = int(mnist.train.num_examples / mini_batch_size)

print ("NN: Hidden1:", hidden1_size,", Hidden2:", hidden2_size,
       ", epochs:", epochs, ", mini batch size:", mini_batch_size,
       " (batches:", mini_batches, "), learning_rate:", learning_rate,
       ", keep_prob_input:", keep_prob_input,
       ", keep_prob_hidden:", keep_prob_hidden, sep='')

for epoch in range(epochs):
    avg_cost = 0.
    for _ in range(mini_batches):
        batch_inputs, batch_labels = mnist.train.next_batch(mini_batch_size)
        _, c = s.run([train, cost], feed_dict={input: batch_inputs, 
                                               label: batch_labels,
                                               tf_keep_prob_input: keep_prob_input,
                                               tf_keep_prob_hidden: keep_prob_hidden})
        
        avg_cost += c / mini_batches
        
    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print(" Accuracy VALIDATION:", s.run(accuracy, feed_dict={input: mnist.validation.images, label: mnist.validation.labels, tf_keep_prob_input: 1.0, tf_keep_prob_hidden: 1.0}))
    print(" Accuracy TEST      :", s.run(accuracy, feed_dict={input: mnist.test.images, label: mnist.test.labels, tf_keep_prob_input: 1.0, tf_keep_prob_hidden: 1.0}))
    print(" Accuracy TRAIN     :", s.run(accuracy, feed_dict={input: mnist.train.images, label: mnist.train.labels, tf_keep_prob_input: 1.0, tf_keep_prob_hidden: 1.0}))
    

NN: Hidden1:500, Hidden2:250, epochs:2500, mini batch size:50 (batches:1100), learning_rate:0.0005, keep_prob_input:0.8, keep_prob_hidden:0.5
Epoch: 0001 cost= 0.781628936
 Accuracy VALIDATION: 0.929
 Accuracy TEST      : 0.9237
 Accuracy TRAIN     : 0.920982
Epoch: 0002 cost= 0.397829951
 Accuracy VALIDATION: 0.9492
 Accuracy TEST      : 0.9445
 Accuracy TRAIN     : 0.944109
Epoch: 0003 cost= 0.311086226
 Accuracy VALIDATION: 0.9612
 Accuracy TEST      : 0.9544
 Accuracy TRAIN     : 0.956055
Epoch: 0004 cost= 0.260779659
 Accuracy VALIDATION: 0.9638
 Accuracy TEST      : 0.9578
 Accuracy TRAIN     : 0.961164
Epoch: 0005 cost= 0.230144301
 Accuracy VALIDATION: 0.97
 Accuracy TEST      : 0.9656
 Accuracy TRAIN     : 0.9674
Epoch: 0006 cost= 0.210050854
 Accuracy VALIDATION: 0.9692
 Accuracy TEST      : 0.9643
 Accuracy TRAIN     : 0.9686
Epoch: 0007 cost= 0.194726689
 Accuracy VALIDATION: 0.9742
 Accuracy TEST      : 0.9701
 Accuracy TRAIN     : 0.974436
Epoch: 0008 cost= 0.176370319
 A

Epoch: 0069 cost= 0.066825465
 Accuracy VALIDATION: 0.9828
 Accuracy TEST      : 0.9822
 Accuracy TRAIN     : 0.995055
Epoch: 0070 cost= 0.067689400
 Accuracy VALIDATION: 0.9854
 Accuracy TEST      : 0.9841
 Accuracy TRAIN     : 0.996364
Epoch: 0071 cost= 0.065535088
 Accuracy VALIDATION: 0.9848
 Accuracy TEST      : 0.9847
 Accuracy TRAIN     : 0.996764
Epoch: 0072 cost= 0.065885185
 Accuracy VALIDATION: 0.984
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.9962
Epoch: 0073 cost= 0.066393073
 Accuracy VALIDATION: 0.985
 Accuracy TEST      : 0.9836
 Accuracy TRAIN     : 0.996655
Epoch: 0074 cost= 0.071971031
 Accuracy VALIDATION: 0.9852
 Accuracy TEST      : 0.9846
 Accuracy TRAIN     : 0.996473
Epoch: 0075 cost= 0.062738450
 Accuracy VALIDATION: 0.9848
 Accuracy TEST      : 0.9839
 Accuracy TRAIN     : 0.9962
Epoch: 0076 cost= 0.065940164
 Accuracy VALIDATION: 0.984
 Accuracy TEST      : 0.9835
 Accuracy TRAIN     : 0.995891
Epoch: 0077 cost= 0.067285342
 Accuracy VALIDATION: 0.9

 Accuracy TEST      : 0.9853
 Accuracy TRAIN     : 0.997545
Epoch: 0139 cost= 0.053034229
 Accuracy VALIDATION: 0.9852
 Accuracy TEST      : 0.9839
 Accuracy TRAIN     : 0.997745
Epoch: 0140 cost= 0.053925588
 Accuracy VALIDATION: 0.987
 Accuracy TEST      : 0.9852
 Accuracy TRAIN     : 0.9982
Epoch: 0141 cost= 0.052360537
 Accuracy VALIDATION: 0.9854
 Accuracy TEST      : 0.9845
 Accuracy TRAIN     : 0.998055
Epoch: 0142 cost= 0.057990347
 Accuracy VALIDATION: 0.9866
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.998055
Epoch: 0143 cost= 0.057266498
 Accuracy VALIDATION: 0.9852
 Accuracy TEST      : 0.9832
 Accuracy TRAIN     : 0.997327
Epoch: 0144 cost= 0.053687149
 Accuracy VALIDATION: 0.9862
 Accuracy TEST      : 0.9842
 Accuracy TRAIN     : 0.998036
Epoch: 0145 cost= 0.051001862
 Accuracy VALIDATION: 0.9876
 Accuracy TEST      : 0.9838
 Accuracy TRAIN     : 0.997855
Epoch: 0146 cost= 0.054521390
 Accuracy VALIDATION: 0.9862
 Accuracy TEST      : 0.9849
 Accuracy TRAIN     : 

 Accuracy TRAIN     : 0.998909
Epoch: 0208 cost= 0.051393078
 Accuracy VALIDATION: 0.9866
 Accuracy TEST      : 0.9854
 Accuracy TRAIN     : 0.998873
Epoch: 0209 cost= 0.048623105
 Accuracy VALIDATION: 0.9874
 Accuracy TEST      : 0.9845
 Accuracy TRAIN     : 0.998764
Epoch: 0210 cost= 0.047243325
 Accuracy VALIDATION: 0.9882
 Accuracy TEST      : 0.9864
 Accuracy TRAIN     : 0.998855
Epoch: 0211 cost= 0.052061935
 Accuracy VALIDATION: 0.9876
 Accuracy TEST      : 0.9857
 Accuracy TRAIN     : 0.9988
Epoch: 0212 cost= 0.048939063
 Accuracy VALIDATION: 0.9874
 Accuracy TEST      : 0.9861
 Accuracy TRAIN     : 0.998364
Epoch: 0213 cost= 0.052471285
 Accuracy VALIDATION: 0.9874
 Accuracy TEST      : 0.9856
 Accuracy TRAIN     : 0.999036
Epoch: 0214 cost= 0.048687797
 Accuracy VALIDATION: 0.9874
 Accuracy TEST      : 0.9857
 Accuracy TRAIN     : 0.998836
Epoch: 0215 cost= 0.050379592
 Accuracy VALIDATION: 0.9876
 Accuracy TEST      : 0.9858
 Accuracy TRAIN     : 0.998782
Epoch: 0216 cost= 0

Epoch: 0277 cost= 0.045467929
 Accuracy VALIDATION: 0.9864
 Accuracy TEST      : 0.9852
 Accuracy TRAIN     : 0.998782
Epoch: 0278 cost= 0.048134249
 Accuracy VALIDATION: 0.987
 Accuracy TEST      : 0.9853
 Accuracy TRAIN     : 0.998855
Epoch: 0279 cost= 0.046216941
 Accuracy VALIDATION: 0.9854
 Accuracy TEST      : 0.9862
 Accuracy TRAIN     : 0.9992
Epoch: 0280 cost= 0.045584836
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9856
 Accuracy TRAIN     : 0.9992
Epoch: 0281 cost= 0.048408561
 Accuracy VALIDATION: 0.987
 Accuracy TEST      : 0.9861
 Accuracy TRAIN     : 0.999182
Epoch: 0282 cost= 0.049737975
 Accuracy VALIDATION: 0.986
 Accuracy TEST      : 0.9846
 Accuracy TRAIN     : 0.9986
Epoch: 0283 cost= 0.047464076
 Accuracy VALIDATION: 0.986
 Accuracy TEST      : 0.9864
 Accuracy TRAIN     : 0.999255
Epoch: 0284 cost= 0.048880005
 Accuracy VALIDATION: 0.9864
 Accuracy TEST      : 0.9861
 Accuracy TRAIN     : 0.999055
Epoch: 0285 cost= 0.044830858
 Accuracy VALIDATION: 0.986


...

Epoch: 2216 cost= 0.047928878
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9841
 Accuracy TRAIN     : 0.999836
Epoch: 2217 cost= 0.042057865
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 0.9848
 Accuracy TRAIN     : 0.999818
Epoch: 2218 cost= 0.045265789
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9839
 Accuracy TRAIN     : 0.999782
Epoch: 2219 cost= 0.046441355
 Accuracy VALIDATION: 0.987
 Accuracy TEST      : 0.9836
 Accuracy TRAIN     : 0.999709
Epoch: 2220 cost= 0.038898965
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 0.9848
 Accuracy TRAIN     : 0.999745
Epoch: 2221 cost= 0.039977704
 Accuracy VALIDATION: 0.9862
 Accuracy TEST      : 0.9842
 Accuracy TRAIN     : 0.999873
Epoch: 2222 cost= 0.049724510
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9832
 Accuracy TRAIN     : 0.999818
Epoch: 2223 cost= 0.044000249
 Accuracy VALIDATION: 0.9888
 Accuracy TEST      : 0.9849
 Accuracy TRAIN     : 0.999927
Epoch: 2224 cost= 0.040686054
 Accuracy VALIDATIO

 Accuracy VALIDATION: 0.9862
 Accuracy TEST      : 0.9843
 Accuracy TRAIN     : 0.999709
Epoch: 2286 cost= 0.045776254
 Accuracy VALIDATION: 0.9866
 Accuracy TEST      : 0.9843
 Accuracy TRAIN     : 0.999655
Epoch: 2287 cost= 0.044891396
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9854
 Accuracy TRAIN     : 0.999873
Epoch: 2288 cost= 0.047772894
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9839
 Accuracy TRAIN     : 0.999745
Epoch: 2289 cost= 0.040357307
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9847
 Accuracy TRAIN     : 0.999691
Epoch: 2290 cost= 0.046492809
 Accuracy VALIDATION: 0.9864
 Accuracy TEST      : 0.9841
 Accuracy TRAIN     : 0.999782
Epoch: 2291 cost= 0.037832644
 Accuracy VALIDATION: 0.987
 Accuracy TEST      : 0.9847
 Accuracy TRAIN     : 0.9998
Epoch: 2292 cost= 0.045970808
 Accuracy VALIDATION: 0.9886
 Accuracy TEST      : 0.9846
 Accuracy TRAIN     : 0.999836
Epoch: 2293 cost= 0.042873513
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 

 Accuracy TEST      : 0.9835
 Accuracy TRAIN     : 0.999818
Epoch: 2355 cost= 0.044522753
 Accuracy VALIDATION: 0.9882
 Accuracy TEST      : 0.9839
 Accuracy TRAIN     : 0.999873
Epoch: 2356 cost= 0.043605430
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.999836
Epoch: 2357 cost= 0.042831058
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 0.9837
 Accuracy TRAIN     : 0.9998
Epoch: 2358 cost= 0.043598418
 Accuracy VALIDATION: 0.9864
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.999855
Epoch: 2359 cost= 0.046126905
 Accuracy VALIDATION: 0.986
 Accuracy TEST      : 0.9848
 Accuracy TRAIN     : 0.999855
Epoch: 2360 cost= 0.039981530
 Accuracy VALIDATION: 0.9854
 Accuracy TEST      : 0.984
 Accuracy TRAIN     : 0.999782
Epoch: 2361 cost= 0.049519956
 Accuracy VALIDATION: 0.988
 Accuracy TEST      : 0.9848
 Accuracy TRAIN     : 0.999873
Epoch: 2362 cost= 0.046565278
 Accuracy VALIDATION: 0.9866
 Accuracy TEST      : 0.9846
 Accuracy TRAIN     : 0.

 Accuracy TRAIN     : 0.999855
Epoch: 2424 cost= 0.043021233
 Accuracy VALIDATION: 0.9864
 Accuracy TEST      : 0.9847
 Accuracy TRAIN     : 0.999782
Epoch: 2425 cost= 0.038560718
 Accuracy VALIDATION: 0.9868
 Accuracy TEST      : 0.9838
 Accuracy TRAIN     : 0.999945
Epoch: 2426 cost= 0.043698184
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9854
 Accuracy TRAIN     : 0.999909
Epoch: 2427 cost= 0.043801477
 Accuracy VALIDATION: 0.9852
 Accuracy TEST      : 0.9845
 Accuracy TRAIN     : 0.999782
Epoch: 2428 cost= 0.043652191
 Accuracy VALIDATION: 0.986
 Accuracy TEST      : 0.9849
 Accuracy TRAIN     : 0.999836
Epoch: 2429 cost= 0.046764783
 Accuracy VALIDATION: 0.9846
 Accuracy TEST      : 0.9828
 Accuracy TRAIN     : 0.999745
Epoch: 2430 cost= 0.040499739
 Accuracy VALIDATION: 0.9856
 Accuracy TEST      : 0.9845
 Accuracy TRAIN     : 0.999836
Epoch: 2431 cost= 0.042380229
 Accuracy VALIDATION: 0.9856
 Accuracy TEST      : 0.9846
 Accuracy TRAIN     : 0.999927
Epoch: 2432 cost= 

Epoch: 2493 cost= 0.045202884
 Accuracy VALIDATION: 0.9862
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.999764
Epoch: 2494 cost= 0.047911374
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 0.9861
 Accuracy TRAIN     : 0.999873
Epoch: 2495 cost= 0.043952439
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9838
 Accuracy TRAIN     : 0.999836
Epoch: 2496 cost= 0.045965250
 Accuracy VALIDATION: 0.9858
 Accuracy TEST      : 0.9844
 Accuracy TRAIN     : 0.999782
Epoch: 2497 cost= 0.042704218
 Accuracy VALIDATION: 0.9874
 Accuracy TEST      : 0.985
 Accuracy TRAIN     : 0.999855
Epoch: 2498 cost= 0.041251732
 Accuracy VALIDATION: 0.9872
 Accuracy TEST      : 0.9854
 Accuracy TRAIN     : 0.999873
Epoch: 2499 cost= 0.041126649
 Accuracy VALIDATION: 0.9848
 Accuracy TEST      : 0.9847
 Accuracy TRAIN     : 0.999836
Epoch: 2500 cost= 0.039398716
 Accuracy VALIDATION: 0.9852
 Accuracy TEST      : 0.9837
 Accuracy TRAIN     : 0.999636
